In [117]:
import requests
import base64
import json

# 1. Access Token 발급
def get_access_token(client_id, client_secret):
    auth_str = f"{client_id}:{client_secret}"
    b64_auth_str = base64.b64encode(auth_str.encode()).decode()
    token_url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': f'Basic {b64_auth_str}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {'grant_type': 'client_credentials'}
    res = requests.post(token_url, headers=headers, data=data)

    if res.status_code == 200:
        return res.json().get('access_token')
    else:
        print(f"❌ 토큰 발급 실패: {res.status_code}")
        print(res.json())
        return None

# 2. 감성 벡터 → Spotify 파라미터 변환
def emotion_to_spotify_params(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        contributions = json.load(f)

    emotion_to_spotify_feature = {
        'Comfort': 'acousticness',
        'Brightness': 'valence',
        'Clarity': 'instrumentalness',
        'Energy': 'energy',
        'Warmth': 'tempo',
        'Sadness': 'liveness',
        'Dreamy': 'speechiness'
    }

    safe_bounds = {
        'valence': (0.1, 0.9),
        'acousticness': (0.1, 0.9),
        'energy': (0.1, 0.9),
        'instrumentalness': (0.0, 0.9),
        'liveness': (0.1, 0.8),
        'speechiness': (0.0, 0.8)
    }

    params = {
        'seed_genres': 'pop',
        'seed_artists': '4NHQUGzhtTLFvgF5SZesLK',  # Tove Lo
        'limit': 5
    }

    for emotion, value in contributions.items():
        feature = emotion_to_spotify_feature.get(emotion)
        if not feature:
            continue
        if feature == 'tempo':
            params['target_tempo'] = round(value * 120 + 60, 2)
        else:
            low, high = safe_bounds.get(feature, (0.0, 1.0))
            params[f'target_{feature}'] = round(min(max(value, low), high), 3)

    return params

# 3. 추천 API 호출 (3단계 fallback)
def get_recommendations(access_token, spotify_params, fallback_stage=1):
    url = "https://api.spotify.com/v1/recommendations"
    headers = {'Authorization': f'Bearer {access_token}'}
    res = requests.get(url, headers=headers, params=spotify_params)

    if res.status_code == 200:
        tracks = res.json().get('tracks', [])
        if tracks:
            print("🎵 추천된 곡 리스트:")
            for i, track in enumerate(tracks, 1):
                name = track['name']
                artists = ', '.join(artist['name'] for artist in track['artists'])
                url = track['external_urls']['spotify']
                print(f"{i}. 🎶 {name} - {artists}\n   🔗 {url}")
        else:
            print("😢 추천된 곡이 없습니다.")
    else:
        # fallback 1 → 2단계로 축소
        if fallback_stage == 1:
            print("⚠️ 1차 실패. 조건 축소(valence/acousticness) 후 재시도...\n")
            reduced_params = {
                'seed_genres': spotify_params.get('seed_genres', 'pop'),
                'seed_artists': spotify_params.get('seed_artists'),
                'limit': spotify_params.get('limit', 5)
            }
            for key in ['target_valence', 'target_acousticness']:
                if key in spotify_params:
                    reduced_params[key] = spotify_params[key]
            return get_recommendations(access_token, reduced_params, fallback_stage=2)

        # fallback 2 → 3단계: 필터 제거
        elif fallback_stage == 2:
            print("⚠️ 2차 실패. 모든 target_* 필터 제거 후 마지막 시도...\n")
            final_params = {
                'seed_genres': spotify_params.get('seed_genres', 'pop'),
                'seed_artists': spotify_params.get('seed_artists'),
                'limit': spotify_params.get('limit', 5)
            }
            return get_recommendations(access_token, final_params, fallback_stage=3)

        # fallback 3 → 완전 실패
        else:
            print(f"❌ 최종 추천 실패: {res.status_code}")
            try:
                print("🔍 에러 상세:", res.json())
            except:
                print("⚠️ 응답 본문 파싱 실패. 원문 출력:")
                print(res.text)

# 4. 실행
def main():
    client_id = '293c9acc8f714b2982f92b82420a2e8f'
    client_secret = '1fc8ca516b124ea7ba1b1582b0c951da'
    json_path = 'emotion_contributions_Gunsan_20250511.json'

    access_token = get_access_token(client_id, client_secret)
    if not access_token:
        return

    spotify_params = emotion_to_spotify_params(json_path)
    print("🎯 요청 파라미터:", spotify_params)

    get_recommendations(access_token, spotify_params)

# 실행
main()


🎯 요청 파라미터: {'seed_genres': 'pop', 'seed_artists': '4NHQUGzhtTLFvgF5SZesLK', 'limit': 5, 'target_tempo': 75.84, 'target_acousticness': 0.162, 'target_valence': 0.15, 'target_energy': 0.1, 'target_instrumentalness': 0.145}
⚠️ 1차 실패. 조건 축소(valence/acousticness) 후 재시도...

⚠️ 2차 실패. 모든 target_* 필터 제거 후 마지막 시도...

❌ 최종 추천 실패: 404
⚠️ 응답 본문 파싱 실패. 원문 출력:



In [115]:
# 매우 간단한 성공 테스트용 예시
test_params = {
    'seed_artists': '6eUKZXaKkcviH0Ku9w2n3V',
    'limit': 5
}
res = requests.get("https://api.spotify.com/v1/recommendations", headers={'Authorization': f'Bearer {access_token}'}, params=test_params)
print(res.status_code)
print(res.json())


401
{'error': {'status': 401, 'message': 'Invalid access token'}}


In [121]:
client_id = '293c9acc8f714b2982f92b82420a2e8f'
client_secret = '1fc8ca516b124ea7ba1b1582b0c951da'

access_token = get_access_token(client_id, client_secret)

test_params = {
    'seed_artists': '6eUKZXaKkcviH0Ku9w2n3V',  # Ed Sheeran
    'limit': 5
}

headers = {'Authorization': f'Bearer {access_token}'}
res = requests.get("https://api.spotify.com/v1/recommendations", headers=headers, params=test_params)

print("Status:", res.status_code)
print("Result:", res.json())


Status: 404


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [123]:
print("Status:", res.status_code)
print("Raw response:", res.text)


Status: 404
Raw response: 


In [125]:
token = get_access_token(client_id, client_secret)
print("Access Token:", token)


Access Token: BQBnYHiNStwemtlhILLc7rKir6CgYlYCzUTh1hgf2mM5RC2dj-Y8qyy3SdRy6qhm4OVdtYfS6fEi1ckbVkZtSMqD6CtIJH8TMRbOyth2LlbqJJbP4O04uXyi9Qh7QuQxGqz2XTFRKTo


In [129]:
client_id = '293c9acc8f714b2982f92b82420a2e8f'
client_secret = '1fc8ca516b124ea7ba1b1582b0c951da'

access_token = get_access_token(client_id, client_secret)
headers = {'Authorization': f'Bearer {access_token}'}
test_params = {
    'seed_artists': '4NHQUGzhtTLFvgF5SZesLK',  # Tove Lo
    'seed_tracks': '0c6xIDDpzE81m2q797ordA',   # I Took A Pill In Ibiza
    'limit': 5
}

res = requests.get("https://api.spotify.com/v1/recommendations", headers=headers, params=test_params)
print("Status:", res.status_code)
print("Response:", res.json() if res.status_code == 200 else res.text)
print("Access Token:", access_token)

if access_token:
    test_params = {
        'seed_artists': '6eUKZXaKkcviH0Ku9w2n3V',  # Ed Sheeran
        'limit': 5
    }

    headers = {'Authorization': f'Bearer {access_token}'}
    res = requests.get("https://api.spotify.com/v1/recommendations", headers=headers, params=test_params)

    print("Status:", res.status_code)
    print("Raw Response:", res.text)  # HTML, error, 또는 아무 것도 없을 수 있음

    try:
        print("JSON Parsed:", res.json())
    except Exception as e:
        print("⚠️ JSON 파싱 실패:", e)


Status: 404
Response: 
Access Token: BQBA1m1RuLsRG_LjvaUbgtiO1-uKH1Zj7n_Ju3F5w1LjKt8PMsFz9kH5zAHKpyrgnti3vE6_-z_RICvm3DMD9-XGFketF22LsIZYqKrKSuKnQSAwtNGj7NBlcUvSp2dmvgvir2mir5g
Status: 404
Raw Response: 
⚠️ JSON 파싱 실패: Expecting value: line 1 column 1 (char 0)
